In [4]:
import subprocess

# Run the gradlew.bat command with the specified arguments
result = subprocess.run(
    [
        r"D:\Repos\Minecraft\SFM\repos\SuperFactoryManager 1.19.2\gradlew.bat",
        "runProjectEverythingLister",
        "--quiet",
        "--daemon",
        "--offline",
    ],
    cwd=r"D:\Repos\Minecraft\SFM\repos\SuperFactoryManager 1.19.2",
    capture_output=True,
    text=True,
)
assert result.stderr == "", result.stderr
output = result.stdout.splitlines()[1:]
output[:5]

['File: src\\datagen\\java\\ca\\teamdman\\sfm\\datagen\\SFMBlockStatesAndModels.java',
 '  Class: SFMBlockStatesAndModels',
 '        @Override',
 '1       protected void registerStatesAndModels()',
 'File: src\\datagen\\java\\ca\\teamdman\\sfm\\datagen\\SFMBlockTags.java']

In [18]:
methods = []
for line in output:
    try:
        index, sig = line.split(maxsplit=1)
        index = int(index)
    except ValueError:
        continue
    assert len(methods) == index - 1, (methods, index)
    methods.append(sig)
methods

['protected void registerStatesAndModels()',
 'protected void addTags()',
 'public String getName()',
 'public static void onGather(GatherDataEvent event)',
 'protected void registerModels()',
 'private void justParent(RegistryObject<? extends Item> item, RegistryObject<? extends Block> block)',
 'private void justParent(RegistryObject<? extends Item> item, RegistryObject<? extends Block> block, String extra)',
 'private void basicItem(RegistryObject<? extends Item> item)',
 'protected void addTranslations()',
 'protected List<Pair<Supplier<Consumer<BiConsumer<ResourceLocation, LootTable.Builder>>>, LootContextParamSet>> getTables()',
 'protected void validate(Map<ResourceLocation, LootTable> map, ValidationContext tracker)',
 'protected void addTables()',
 'protected Iterable<Block> getKnownBlocks()',
 'protected void buildCraftingRecipes(Consumer<FinishedRecipe> consumer)',
 'private void addPrintingPressRecipe(Consumer<FinishedRecipe> consumer, ResourceLocation id, Ingredient form, 

In [20]:
nl = "\n"
prompt = f"""
We are going to add another block to a Minecraft mod.
You are selecting which existing files should be inspected for context on how to interact with the repo.
You must select AT LEAST ten method indices and AT LEAST ten class names.

{nl.join(output)}
"""
print("\n".join(prompt.splitlines()[:10]), "\n...")


We are going to add another block to a Minecraft mod.
You are selecting which existing files should be inspected for context on how to interact with the repo.
You must select AT LEAST ten method indices and AT LEAST ten class names.

File: src\datagen\java\ca\teamdman\sfm\datagen\SFMBlockStatesAndModels.java
  Class: SFMBlockStatesAndModels
        @Override
1       protected void registerStatesAndModels()
File: src\datagen\java\ca\teamdman\sfm\datagen\SFMBlockTags.java 
...


In [21]:
from openai import OpenAI
from pydantic import BaseModel, ValidationError, model_validator
import instructor

# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)
model = "qwen2.5:iq4"

In [24]:
from typing import List


class RelevantMethods(BaseModel):
    method_indices: List[int]
    class_names: List[str]

    @model_validator(mode="after")
    def check_indices(cls, v):
        if len(v.method_indices) < 10:
            raise ValueError("At least ten methods must be selected")
        return v
    
    @model_validator(mode="after")
    def check_class_names(cls, v):
        if len(v.class_names) < 10:
            raise ValueError("At least ten methods must be selected")
        return v

resp = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    response_model=RelevantMethods,
)

print(resp)

method_indices=[1, 2, 3, 4, 6, 9, 10, 15, 16, 18, 20, 21, 23, 24, 26, 27, 37, 41, 45, 47, 48, 53, 57, 59, 60, 61, 73, 78, 80, 84, 86, 89, 92, 93, 94, 95, 97, 98, 100, 101, 105, 108, 111, 112, 117, 119, 163, 164, 170, 172, 173, 175, 176, 178, 179, 180, 184, 189, 191, 192, 194, 195, 198, 199, 203, 206, 210, 213, 222, 223, 237, 241] class_names=['SFMBlockStatesAndModels', 'SFMBlockTags', 'SFMBlockTags', 'SFMDatagen', 'SFMItemModels', 'SFMLanguageProvider', 'SFMLootTable', 'SFMAppliedEnergisticsCompatGameTests', 'SFMIndustrialForegoingCompatGameTests', 'SFMMekanismCompatGameTests', 'ManagerScreen', 'SFMLoader', 'MyMultiLineEditBox', 'ContainerScreenInspectorHandler', 'LabelGunScrollSwitcher', 'SFMJEIPlugin', 'ProgramSyntaxHighlightingHelper', 'ExtendedButtonWithTooltip', 'LogsScreen', 'LogsScreen', 'MoveSlotTestBase', 'ManagerStateUpdateTests', 'CauldronLavaMoveTests', 'GameTestMethodUtils.MoveOnlyIfDifferentGameTestConditionedOnBlockEntityTickResults', 'SFMLoader', 'LabelGunEditorHandlerC

In [26]:
chosen = [methods[i - 1] for i in resp.method_indices]
chosen

['protected void registerStatesAndModels()',
 'protected void addTags()',
 'public String getName()',
 'public static void onGather(GatherDataEvent event)',
 'private void justParent(RegistryObject<? extends Item> item, RegistryObject<? extends Block> block)',
 'protected void addTranslations()',
 'protected List<Pair<Supplier<Consumer<BiConsumer<ResourceLocation, LootTable.Builder>>>, LootContextParamSet>> getTables()',
 'private void addPrintingPressRecipe(Consumer<FinishedRecipe> consumer, ResourceLocation id, Ingredient form, Ingredient ink, Ingredient paper)',
 'public static void ae2_inscribers(GameTestHelper helper)',
 'public static void industrialforegoing_blackhole_some(GameTestHelper helper)',
 'public static void mek_chemtank_infusion_empty(GameTestHelper helper)',
 'public static void mek_chemtank_infusion_some(GameTestHelper helper)',
 'public static void mek_bin_empty(GameTestHelper helper)',
 'public static void mek_bin_some(GameTestHelper helper)',
 'public static void